In [1]:
# Load Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

import warnings

warnings.filterwarnings("ignore")

from typing import List

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasRegressor
from keras.constraints import MaxNorm

import keras

import model_prep

rootpath = ".."

step_back = 6  # window size = 6*5 = 30 mins

2023-08-01 20:22:12.760755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def run_grid_search(building_name, tower_number, season, param_grid, use_delta=True, train_percentage=0.75, shuffle_seed=42):
    features = [f'{building_name}_Tower_{tower_number} enteringWaterTemp', 
                    f'{building_name}_Tower_{tower_number} outdoorAirDryBulb', 
                    f'{building_name}_Tower_{tower_number} outdoorAirWetBulb', 
                    f'{building_name}_Tower_{tower_number} vfdPercent', 
                    f'{building_name}_Tower_{tower_number} fanA_vfdPower', 
                    f'{building_name}_Tower_{tower_number} fanB_vfdPower', 
                    f'{building_name}_Tower_{tower_number} dayOfWeek', 
                    f'{building_name}_Tower_{tower_number} hourOfDay', 
                    f'{building_name}_Tower_{tower_number} efficiency']
    target = f'{building_name}_Tower_{tower_number} leavingWaterTemp'

    """
    1. Convert data into a model-compatible shape
    """

    lstm_df, first_temp = model_prep.create_preprocessed_lstm_df(
        building_name=building_name,
        tower_number=tower_number,
        features=features,
        target=target,
        season=season,
        use_delta=use_delta,
    )
    if not season:
        season = "allyear"

    """
    2. Split data into training and testing sets
    """

    X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
    y = lstm_df[f"{target}(t)"]  # only have target column

    # split into input and outputs
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=(1 - train_percentage), shuffle=True, random_state=shuffle_seed
    )

    # scale feature data
    scaler = MinMaxScaler().fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    """
    3. Get timestepped data as a 3D vector
    """
    vec_X_train = model_prep.df_to_3d(
        lstm_dtframe=X_train, num_columns=len(features) + 1, step_back=step_back
    )
    vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )

    vec_y_train = y_train.values
    vec_y_test = y_test.values

    """
    4. Create and Train model
    """
    # Create a function that builds the Keras model
    def create_model():
        dropout_rate = 0.0
        weight_constraint = 2.0
        lstmcells = 32
        activation = 'tanh'
        optimizer = "Adamax"

        model = keras.models.Sequential()
        model.add(
            keras.layers.LSTM(
                lstmcells,
                input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
                kernel_constraint=MaxNorm(weight_constraint),
                recurrent_dropout=dropout_rate,
                activation=activation
            )
        )
        model.add(keras.layers.Dense(1))
        model.compile(loss='mse', optimizer=optimizer)
        return model

    # Create a KerasClassifier
    model = KerasRegressor(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

    # Create GridSearchCV and perform the grid search
    grid_search = GridSearchCV(model, param_grid, cv=3)
    grid_result = grid_search.fit(vec_X_train, vec_y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [3]:
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
neurons = [6, 16, 32, 64, 128]
# lstmcells = [1, 6, 16, 32, 64]
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# param_grid = dict(optimizer=optimizer)
# param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)
param_grid = dict(model__activation=activation)

```
batch size: 10
optimizer: Adamax
dropout rate: 0.0
weight constraints: 2.0
neurons:
lstm cells:
activation: tanh
```

In [4]:
run_grid_search(building_name="ESB", tower_number=1, season="summer", param_grid=param_grid)

2023-08-01 20:22:16.510475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Best: 0.994509 using {'model__activation': 'softsign'}
0.916583 (0.001123) with: {'model__activation': 'softmax'}
0.991455 (0.001383) with: {'model__activation': 'softplus'}
0.994509 (0.001657) with: {'model__activation': 'softsign'}
0.993416 (0.000852) with: {'model__activation': 'relu'}
0.994260 (0.000149) with: {'model__activation': 'tanh'}
0.990714 (0.000815) with: {'model__activation': 'sigmoid'}
0.989893 (0.000776) with: {'model__activation': 'hard_sigmoid'}
0.994246 (0.001111) with: {'model__activation': 'linear'}


In [5]:
run_grid_search(building_name="Kissam", tower_number=1, season="summer", param_grid=param_grid)

Best: 0.995270 using {'model__activation': 'tanh'}
0.973592 (0.000419) with: {'model__activation': 'softmax'}
0.993263 (0.000995) with: {'model__activation': 'softplus'}
0.994966 (0.000445) with: {'model__activation': 'softsign'}
0.994501 (0.000514) with: {'model__activation': 'relu'}
0.995270 (0.000473) with: {'model__activation': 'tanh'}
0.992127 (0.000800) with: {'model__activation': 'sigmoid'}
0.992863 (0.000752) with: {'model__activation': 'hard_sigmoid'}
0.994847 (0.000702) with: {'model__activation': 'linear'}


In [ ]:
run_grid_search(building_name="ESB", tower_number=2, season="summer", param_grid=param_grid)

In [ ]:
run_grid_search(building_name="Kissam", tower_number=2, season="summer", param_grid=param_grid)

In [ ]:
run_grid_search(building_name="ESB", tower_number=1, season="fall", param_grid=param_grid)

In [ ]:
run_grid_search(building_name="ESB", tower_number=2, season="fall", param_grid=param_grid)

In [ ]:
run_grid_search(building_name="Kissam", tower_number=1, season="fall", param_grid=param_grid)

In [ ]:
run_grid_search(building_name="Kissam", tower_number=2, season="fall", param_grid=param_grid)